In [ ]:
import numpy as np
#import os

import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt

from astropy.io import fits

from setup import *
from FullFrame import FullFrame

##--------------------------------------------------------------------------##
##             NECESSARY USER INPUTS BELOW....                              ##
##--------------------------------------------------------------------------##

##################
###   WASP4b   ###
##################
obs_date='ut20150819'                           #observation date
obj_name='Wasp4'                                #object name   
midtime=['2015-08-19T07:43:00.00']                 #time of midtransit, WAsp-4b

###################
###   WASP52b   ###
###################
#obs_date='ut20160811'
#obs_date='ut20160922'
#obj_name='Wasp52'

#midtime=['2016-08-11T04:51:00']               #time of midtransit, Wasp-52b, Transit1
#midtime=['2016-09-22T04:44:00']               #time of midtransit, Wasp-52b, Transit2

##################
###   Hats8b   ###
##################
#obs_date='ut20170723'                           #observation date
#obs_date='ut20170828'
#obj_name='Hats8'                                #object name  

#midtime=['2017-07-24T05:28:01.056']                 #time of midtransit, Hats-8b, Transit 1
#midtime=['2017-08-29T01:36:04.608']                 #time of midtransit, Hats-8b Transit 2


path='/Volumes/ermay_ext/Magellan/'+obs_date+'/'
data_path=path+obj_name+'_Spec/'      #path to where your data is saved
flat_path=path+obj_name+'_Flats/'     #path to where your SLIT flats are saved
DARKS=path+obj_name+'_Darks/'     #path to where your darks are saved
FLATS=path+'Full_Flats/'             #path to where the full field flats are saved 
WAVE=path+obj_name+'_Arcs/'

#SAVEPATH='SaveData/'+obj_name+'/'
SAVEPATH=path+'SaveData_'+obj_name+'/'

wavelength_path=path+'WaveSolution/'



extray=400 #number of extra pixels to extract per side in y direction
extrax=80  #number on the side to find wavelength strip, only for nights with a single mask


binn=1    #binning
#### CHECK BINNING IN SETUP.PY ####
#### if 1x1 binning, xpixels,ypixels should be 2048,4096
#### if 2x2 binning, xpixels,ypixels should be 1024,2048
print xpixels/binn, ypixels/binn

# --------------------------------------------------------------------------- 
# Wavelength Calibration #

Not working in notebook, will eventually update this code to run this way. 

Currently: Run via command line. I'll include instructions for this part soon.

In [ ]:
# First we must extract the wavelength calibration spectrum

# THIS CELL IS FOR OBSERVING RUNS WITH A SINGLE MASK FOR DATA/WAVELENGTH CALIBRATION

from WaveExtract import DetermineSide

#extray=400 #number of extra pixels to extract per side in y direction
#extrax=50

data=DetermineSide(WAVE,extray,extrax,SAVEPATH)
#del data

In [ ]:
from WaveExtract import Extract_wave_left, Extract_wave_right

#extray=400 #number of extra pixels to extract per side in y direction
#extrax=80

lr_arr=['left','right','right','right','right','left','left','left','right'] #W52
#lr_arr=['left','left','right','left','left','right','left','left','left','right','right','left','right'] #W4

#n_obj=13  #W4
#n_wave=8  #W4

n_obj=9
n_wave=12  #W52_1

for i in range(0,n_obj):
    if lr_arr[i]=='left':
        print '------------------------'
        print 'EXTRACTING WAVE FOR: ',i
        print '------------------------'
        run=Extract_wave_left(WAVE,extray,extrax,SAVEPATH,i,n_wave)
    elif lr_arr[i]=='right':
        print '------------------------'
        print 'EXTRACTING WAVE FOR: ',i
        print '------------------------'
        run=Extract_wave_right(WAVE,extray,extrax,SAVEPATH,i,n_wave)

In [ ]:
### For all targets after 2017A

from WaveExtract import Extract_wave_center

n_obj=8
n_wave=5

ARCS_ar=['Ar','He','Ne']
ARCS_ot=np.empty([2*ypixels+ygap,len(ARCS_ar)])

#for a in range(0,len(ARCS_ar)):
#    WAVE=path+obj_name+'_Arcs_'+str(ARCS_ar[a])+'/'
#    data=DetermineSide(WAVE,extray,extrax,SAVEPATH)

for i in range(0,n_obj):
    print '------------------------'
    print 'EXTRACTING WAVE FOR: ',i
    print '------------------------'
    for a in range(0,len(ARCS_ar)):
        WAVE=path+obj_name+'_Arcs_'+str(ARCS_ar[a])+'/'
        print '*****', ARCS_ar[a], '*****'
        run=Extract_wave_center(WAVE,extray,extrax,SAVEPATH,i,n_wave)
        ARCS_ot[:,a]=run
    ARCS_total=np.nansum(ARCS_ot,axis=1)
    #for a in range(0,len(ARCS_ar)):
    #    plt.plot(ARCS_ot[:,a])
    plt.plot(ARCS_total)
    plt.show(block=False)
    np.savez(SAVEPATH+'wavespec_obj'+str(int(i))+'.npz', spec=np.flip(ARCS_total,0))